In [ ]:
#########improved version always second (CR done)
!pip install keras-adamw
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
from keras.optimizers import AdamW
from tensorflow import keras
from keras import layers
from keras import callbacks

path = "/content/drive/MyDrive/dataset/raw data.csv" #/content/drive/MyDrive/dataset/CR.csv
data = pd.read_csv(path)

data = pd.get_dummies(data) #convert categorical variable into dummy variable

y = np.array(data['CR (%)']) #output feature
X = data.drop(['CR (%)', 'PDOs PY (%)', 'I (A)'], axis=1) #input feature - axis=1 refers to the column

X_list = list(X.columns) # Saving feature names for later use

from sklearn.preprocessing import StandardScaler #standardscaler for normalization (Jinesh et al., 2023)
scaler = StandardScaler()

y = y.reshape(-1, 1) #normalizing y
scaler.fit(y)
y = scaler.transform(y)

scaler.fit(X) #normalizing X
X = scaler.transform(X)

X = np.array(X) # Convert to numpy array

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

#imputation for missing value using kNN algo
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10) #create an instance of KNNImputer class, k is desired number of neighbours

#perform imputation only on X because y no missing values
imp_train_X = imputer.fit_transform(train_X)
imp_test_X = imputer.fit_transform(test_X)

input_shape = [imp_train_X.shape[1]]


model = keras.Sequential([
    layers.Dense(125, activation='relu', input_shape=input_shape),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(125, activation='relu'),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(125, activation='relu'),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(1)
])

# adamw = AdamW(lr=0.0001, weight_decay=1e-5)
# model.compile(optimizer=adamw, loss='categorical_crossentropy', metrics=['accuracy'])

model.compile(
optimizer='adam',
loss='mae',
)

history = model.fit(
  imp_train_X, train_y,
  validation_data=(imp_test_X,test_y),
  batch_size=3,
  epochs=1000,
)

#r2 score for training set
y_pred = model.predict(imp_train_X)
y_true = train_y
score = r2_score(y_true,y_pred)
print("Training score: ", score)

pred = model.predict(imp_test_X)

print("Mean Absolute Error: " + str(mean_absolute_error(test_y, pred)))
print("MSE: " + str(mean_squared_error(pred, test_y)))

RMSE = sqrt(mean_squared_error(pred, test_y))

print("RMSE: " + str(RMSE))
print("R2 score: " + str(r2_score(test_y, pred)))


Training X Shape: (312, 30)
Training y Shape: (312, 1)
Testing X Shape: (134, 30)
Testing y Shape: (134, 1)
Epoch 1/1000
104/104 [==============================] - 8s 12ms/step - loss: 0.7513 - val_loss: 0.5455
Epoch 2/1000
104/104 [==============================] - 1s 7ms/step - loss: 0.5759 - val_loss: 0.4211
Epoch 3/1000
104/104 [==============================] - 1s 7ms/step - loss: 0.5109 - val_loss: 0.3954
Epoch 4/1000
104/104 [==============================] - 1s 8ms/step - loss: 0.4771 - val_loss: 0.4356
Epoch 5/1000
104/104 [==============================] - 1s 7ms/step - loss: 0.4444 - val_loss: 0.2861
Epoch 6/1000
104/104 [==============================] - 1s 8ms/step - loss: 0.4249 - val_loss: 0.2828
Epoch 7/1000
104/104 [==============================] - 1s 9ms/step - loss: 0.4020 - val_loss: 0.3158
Epoch 8/1000
104/104 [==============================] - 1s 10ms/step - loss: 0.3626 - val_loss: 0.2949
Epoch 9/1000
104/104 [==============================] - 1s 13ms/step - los

In [ ]:
#sometime wait for the third will not be too bad!

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

from tensorflow import keras
from keras import layers
from keras import callbacks

path = "/content/drive/MyDrive/dataset/PDOs PY copy.csv"
data = pd.read_csv(path)

data = pd.get_dummies(data) #convert categorical variable into dummy variable

y = np.array(data['PY']) #output feature
X = data.drop(['PY'], axis=1) #input feature - axis=1 refers to the column

X_list = list(X.columns) # Saving feature names for later use

from sklearn.preprocessing import StandardScaler #standardscaler for normalization (Jinesh et al., 2023)
scaler = StandardScaler()

y = y.reshape(-1, 1) #normalizing y
scaler.fit(y)
y = scaler.transform(y)

scaler.fit(X) #normalizing X
X = scaler.transform(X)

X = np.array(X) # Convert to numpy array

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 42)
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

#imputation for missing value using kNN algo
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10) #create an instance of KNNImputer class, k is desired number of neighbours

#perform imputation on on X because y no missing values
imp_train_X = imputer.fit_transform(train_X)
imp_test_X = imputer.fit_transform(test_X)

input_shape = [imp_train_X.shape[1]]

model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=input_shape),
    #layers.BatchNormalization(),
    #layers.Dropout(0.3),
    layers.Dense(512, activation='relu'),
    #layers.BatchNormalization(),
    #layers.Dropout(0.3),
    #layers.Dense(512, activation='relu'),
    #layers.BatchNormalization(),
    #layers.Dropout(0.3),
    layers.Dense(1)
])

model.compile(
optimizer='adam',
loss='mae')

r = model.fit(imp_train_X, train_y,
              validation_data=(imp_test_X,test_y),
              batch_size=64,
              epochs=50)

#r2 score for training set (true, predicted)
test_pred = model.predict(imp_test_X)
train_pred = model.predict(imp_train_X)

score = r2_score(train_y, train_pred)
print("Training R2 score: ", score)

print("Mean Absolute Error: " + str(mean_absolute_error(test_y, test_pred)))
print("MSE: " + str(mean_squared_error(test_y, test_pred)))

RMSE = sqrt(mean_squared_error(test_y, test_pred))

print("RMSE: " + str(RMSE))
print("R2 score: " + str(r2_score(test_y, test_pred)))



Training X Shape: (242, 31)
Training y Shape: (242, 1)
Testing X Shape: (81, 31)
Testing y Shape: (81, 1)
Epoch 1/50
4/4 [==============================] - 1s 61ms/step - loss: 0.6235 - val_loss: 0.4573
Epoch 2/50
4/4 [==============================] - 0s 24ms/step - loss: 0.4789 - val_loss: 0.4533
Epoch 3/50
4/4 [==============================] - 0s 22ms/step - loss: 0.4495 - val_loss: 0.3864
Epoch 4/50
4/4 [==============================] - 0s 17ms/step - loss: 0.4360 - val_loss: 0.4190
Epoch 5/50
4/4 [==============================] - 0s 23ms/step - loss: 0.3776 - val_loss: 0.4184
Epoch 6/50
4/4 [==============================] - 0s 24ms/step - loss: 0.3927 - val_loss: 0.3544
Epoch 7/50
4/4 [==============================] - 0s 17ms/step - loss: 0.3485 - val_loss: 0.4084
Epoch 8/50
4/4 [==============================] - 0s 23ms/step - loss: 0.3318 - val_loss: 0.3525
Epoch 9/50
4/4 [==============================] - 0s 18ms/step - loss: 0.3292 - val_loss: 0.3082
Epoch 10/50
4/4 [====

In [ ]:
########improved version always second for PDOs PY now ECR
!pip install keras-adamw
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
from keras.optimizers import AdamW
from tensorflow import keras
from keras import layers
from keras import callbacks

path = "/content/drive/MyDrive/dataset/raw data.csv" #/content/drive/MyDrive/dataset/CR.csv
data = pd.read_csv(path)

data = pd.get_dummies(data) #convert categorical variable into dummy variable

y = np.array(data['PDOs PY (%)']) #output feature
X = data.drop(['CR (%)', 'ECR PY (%)', 'PDOs PY (%)', 'POHs PY (%)', 'V'], axis=1) #input feature - axis=1 refers to the column

X_list = list(X.columns) # Saving feature names for later use

from sklearn.preprocessing import StandardScaler #standardscaler for normalization (Jinesh et al., 2023)
scaler = StandardScaler()

y = y.reshape(-1, 1) #normalizing y
scaler.fit(y)
y = scaler.transform(y)

scaler.fit(X) #normalizing X
X = scaler.transform(X)

X = np.array(X) # Convert to numpy array

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
print('Training X Shape:', train_X.shape)
print('Training y Shape:', train_y.shape)
print('Testing X Shape:', test_X.shape)
print('Testing y Shape:', test_y.shape)

#imputation for missing value using kNN algo
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=10) #create an instance of KNNImputer class, k is desired number of neighbours

#perform imputation only on X because y no missing values
imp_train_X = imputer.fit_transform(train_X)
imp_test_X = imputer.fit_transform(test_X)

input_shape = [imp_train_X.shape[1]]


model = keras.Sequential([
    layers.Dense(125, activation='relu', input_shape=input_shape),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(125, activation='relu'),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(125, activation='relu'),
    #layers.BatchNormalization(),
    layers.Dropout(0.25),
    layers.Dense(1)
])

# adamw = AdamW(lr=0.0001, weight_decay=1e-5)
# model.compile(optimizer=adamw, loss='categorical_crossentropy', metrics=['accuracy'])

model.compile(
optimizer='adam',
loss='mae',
)

history = model.fit(
  imp_train_X, train_y,
  validation_data=(imp_test_X,test_y),
  batch_size=3,
  epochs=1000,
)

#r2 score for training set
y_pred = model.predict(imp_train_X)
y_true = train_y
score = r2_score(y_true,y_pred)
print("Training score: ", score)

pred = model.predict(imp_test_X)

print("Mean Absolute Error: " + str(mean_absolute_error(test_y, pred)))
print("MSE: " + str(mean_squared_error(pred, test_y)))

RMSE = sqrt(mean_squared_error(pred, test_y))

print("RMSE: " + str(RMSE))
print("R2 score: " + str(r2_score(test_y, pred)))

Training X Shape: (312, 30)
Training y Shape: (312, 1)
Testing X Shape: (134, 30)
Testing y Shape: (134, 1)
Epoch 1/1000
104/104 [==============================] - 4s 12ms/step - loss: 0.6112 - val_loss: 0.6210
Epoch 2/1000
104/104 [==============================] - 1s 8ms/step - loss: 0.5597 - val_loss: 0.6350
Epoch 3/1000
104/104 [==============================] - 1s 10ms/step - loss: 0.5212 - val_loss: 0.5590
Epoch 4/1000
104/104 [==============================] - 1s 6ms/step - loss: 0.4804 - val_loss: 0.5561
Epoch 5/1000
104/104 [==============================] - 1s 5ms/step - loss: 0.4931 - val_loss: 0.5687
Epoch 6/1000
104/104 [==============================] - 1s 6ms/step - loss: 0.4672 - val_loss: 0.5354
Epoch 7/1000
104/104 [==============================] - 1s 6ms/step - loss: 0.4278 - val_loss: 0.4919
Epoch 8/1000
104/104 [==============================] - 1s 5ms/step - loss: 0.4252 - val_loss: 0.4744
Epoch 9/1000
104/104 [==============================] - 1s 5ms/step - loss